In [20]:
import pytesseract
from pdf2image import convert_from_path
from transformers import AutoTokenizer, MBartForConditionalGeneration, pipeline
import os


tokenizer = AutoTokenizer.from_pretrained('./ru-mbart-large-summ')
model = MBartForConditionalGeneration.from_pretrained('./ru-mbart-large-summ')

os.environ['TESSDATA_PREFIX'] = '/opt/homebrew/share/tessdata'

pages = convert_from_path('test3.pdf', dpi=300)

extracted_text = ''

for i, page in enumerate(pages):
    text = pytesseract.image_to_string(page, lang='rus+eng')
    extracted_text += f'--- Страница {i+1} ---\n' + text + '\n'

summ = pipeline("summarization", model=model, tokenizer=tokenizer)

res = summ(extracted_text)[0]['summary_text']
res

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

'Министр образования и науки РФ Владимир ПРЕОБРАЖЕНСКИЙ: «Высшее профессиональное образование Москвы — это школа дополнительного образования»'

In [ ]:
import os
import torch
import pytesseract
from pdf2image import convert_from_path
from transformers import AutoTokenizer, MBartForConditionalGeneration, pipeline
import subprocess

# Если используется Tesseract, указываем путь к tessdata
os.environ['TESSDATA_PREFIX'] = '/opt/homebrew/share/tessdata'

# Определяем устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загружаем модель и токенизатор (модель дообучена и сохранена в 'tuned/final_model')
tokenizer = AutoTokenizer.from_pretrained('tuned/final_model')
model = MBartForConditionalGeneration.from_pretrained('tuned/final_model').to(device)

# Функция для обработки .docx (не забудьте установить python-docx: pip install python-docx)
def convert_docx_to_txt(docx_path):
    from docx import Document
    doc = Document(docx_path)
    full_text = [para.text for para in doc.paragraphs]
    text = "\n".join(full_text)
    return text

def convert_doc_to_txt(doc_path):
    try:
        result = subprocess.run(
            ["textutil", "-convert", "txt", "-stdout", doc_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        if result.returncode == 0:
            return result.stdout
        else:
            print("Ошибка:", result.stderr)
            return ""
    except Exception as e:
        print("Ошибка при запуске catdoc:", e)
        return ""

def ocr_pdf(pdf_path):
    """
    Извлекает текст из PDF с помощью OCR.
    """
    pages = convert_from_path(pdf_path, dpi=300)
    extracted_text = ''
    for i, page in enumerate(pages):
        text = pytesseract.image_to_string(page, lang='rus+eng')
        extracted_text += text
    return extracted_text

def get_text_from_file(file_path):
    """
    Извлекает текст из файла с поддержкой форматов .txt, .docx и .pdf.
    """
    _, ext = os.path.splitext(file_path)
    ext = ext.lower()
    
    if ext == ".txt":
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    elif ext == ".docx":
        return convert_docx_to_txt(file_path)
    elif ext == ".doc":
        return convert_doc_to_txt(file_path)
    elif ext == ".pdf":
        return ocr_pdf(file_path)
    else:
        print(f"Неподдерживаемый формат файла: {file_path}")
        return ""

# Создаём pipeline для суммаризации
summ = pipeline(
    "summarization",
    device=0 if torch.cuda.is_available() else -1,
    model=model,
    tokenizer=tokenizer,
    do_sample=False,
    max_length=120,   # максимальная длина генерируемой сводки
    min_length=60
)

def summarize_long_text(text, summ, tokenizer, max_input_length=1024):
    """
    Суммаризует длинный текст, разбивая его на чанки, если он превышает max_input_length токенов.
    1. Текст токенизируется.
    2. Если длина входа <= max_input_length, используется стандартная суммаризация.
    3. Если длина больше — текст делится на чанки, для каждого из которых генерируется сводка.
    4. Итоговые сводки объединяются, и при необходимости проводится финальная суммаризация.
    """
    # Токенизируем текст для определения длины в токенах
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    
    if len(input_ids) <= max_input_length:
        # Если текст не превышает лимит, суммируем напрямую
        return summ(text)[0]['summary_text']
    else:
        # Разбиваем текст на чанки по max_input_length токенов
        chunks = []
        for i in range(0, len(input_ids), max_input_length):
            chunk_ids = input_ids[i:i+max_input_length]
            # Декодируем токены в текст, убирая специальные токены
            chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True)
            chunks.append(chunk_text)
        
        # Генерируем сводку для каждого чанка
        chunk_summaries = []
        for chunk in chunks:
            summary = summ(chunk)[0]['summary_text']
            chunk_summaries.append(summary)
        
        # Объединяем сводки чанков
        combined_summary = " ".join(chunk_summaries)
        
        # Если итоговая сводка всё ещё слишком длинная, можно провести финальную суммаризацию
        combined_input_ids = tokenizer.encode(combined_summary, add_special_tokens=True)
        if len(combined_input_ids) > max_input_length:
            final_summary = summ(combined_summary)[0]['summary_text']
        else:
            final_summary = combined_summary
        
        return final_summary

# Пример использования:
file_path = "proc_data/7.txt"  # путь к вашему файлу (.txt, .docx или .pdf)
text = get_text_from_file(file_path)
res = summarize_long_text(text, summ, tokenizer)
print(res)


'Настоящее положение дел в Российском государственном научно-техническом университете создано в соответствии с законом «О государственной политике Российской Федерации» и уставом Национального исследовательского университета «Высшая школа экономики» (НОУ) «Управление правовой политики НИУ ВШЭ».'

In [14]:
tokenizer_kwargs = {'truncation':True,'max_length':1024}
summ(extracted_text, **tokenizer_kwargs)

[{'summary_text': 'Эксперты «МК» узнали, как будет развиваться отечественный интеллектуальный интеллект'}]